# **ETL Process**

In [74]:
import numpy as np
import pandas as pd

# ignore warnings for cleaner notebook
import warnings
warnings.filterwarnings('ignore')

In [75]:
# reading base dataset
df = pd.read_csv('Project2_Dataset.csv')
df.head()

,ID,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus Involvement,...,Age,National Remoteness Areas,SA4 Name 2021,National LGA Name 2024,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,1,20241115,NSW,12,2024,Friday,4:00,Single,1,No,...,74,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,65_to_74,Weekday,Night
1,2,20241125,NSW,12,2024,Friday,6:15,Single,1,No,...,19,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,17_to_25,Weekday,Day
2,3,20246013,TAS,12,2024,Friday,9:43,Single,1,No,...,33,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,26_to_39,Weekday,Day
3,4,20241002,NSW,12,2024,Friday,10:35,Single,1,No,...,32,Outer Regional Australia,New England and North West,Armidale,National or State Highway,No,No,26_to_39,Weekday,Day
4,5,20243185,QLD,12,2024,Friday,13:00,Single,1,No,...,61,Inner Regional Australia,Toowoomba,Lockyer Valley,National or State Highway,No,No,40_to_64,Weekday,Day


In [76]:
df.columns

Index(['ID', 'Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time',
       'Crash Type', 'Number Fatalities', 'Bus Involvement',
       'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement',
       'Speed Limit', 'Road User', 'Gender', 'Age',
       'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2024',
       'National Road Type', 'Christmas Period', 'Easter Period', 'Age Group',
       'Day of week', 'Time of day'],
      dtype='object')

In [77]:
df = df.drop_duplicates()
df.isna().sum()

ID                               0
Crash ID                         0
State                            0
Month                            0
Year                             0
Dayweek                          0
Time                             0
Crash Type                       0
Number Fatalities                0
Bus Involvement                  0
Heavy Rigid Truck Involvement    0
Articulated Truck Involvement    0
Speed Limit                      0
Road User                        0
Gender                           0
Age                              0
National Remoteness Areas        0
SA4 Name 2021                    0
National LGA Name 2024           0
National Road Type               0
Christmas Period                 0
Easter Period                    0
Age Group                        0
Day of week                      0
Time of day                      0
dtype: int64

In [78]:
for col in df.select_dtypes(exclude='number'):
    print(f"{col}: {df[col].unique()}")

State: ['NSW' 'TAS' 'QLD' 'SA' 'VIC' 'ACT' 'NT' 'WA']
Dayweek: ['Friday' 'Monday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday' 'Wednesday']
Time: ['4:00' '6:15' '9:43' ... '21:54' '5:33' '20:51']
Crash Type: ['Single' 'Multiple']
Bus Involvement: ['No' 'Yes']
Heavy Rigid Truck Involvement: ['No' 'Yes']
Articulated Truck Involvement: ['No' 'Yes']
Road User: ['Driver' 'Passenger' 'Motorcycle rider' 'Pedestrian' 'Pedal cyclist'
 'Motorcycle pillion passenger']
Gender: ['Male' 'Female']
National Remoteness Areas: ['Inner Regional Australia' 'Outer Regional Australia'
 'Major Cities of Australia' 'Very Remote Australia' 'Remote Australia']
SA4 Name 2021: ['Riverina' 'Sydney - Baulkham Hills and Hawkesbury'
 'Launceston and North East' 'New England and North West' 'Toowoomba'
 'Barossa - Yorke - Mid North' 'Darling Downs - Maranoa' 'Hobart'
 'South Australia - South East' 'Sydney - Outer West and Blue Mountains'
 'Capital Region' 'Cairns' 'Illawarra' 'Hunter Valley exc Newcastle'
 'Wide Bay' 'Ce

## **Creating Node and Relationship Tables**
The data has been cleaned (no duplicates, no nulls, no invalid values). The raw dataset will be split into node and relationship tables.

From the above table we can **identify the following entities:**
1. **Crash:** each crash event, using Crash ID as an identifier
2. **Person:** people involved in the fatalities
3. **Location:** geographical attributes of the crash
4. **Vehicle:** involvement of vehicles in the crash
5. **Time:** time period when the crash occurred
6. **Road:** road conditions of the crash

The following **relationships** can also be identified:
1. **KILLED** (Crash $\rightarrow$ Person)
    - **DIED_IN** (Person $\rightarrow$ Crash)
2. **OCCURRED_AT** (Crash $\rightarrow$ Location)
    - **HAS_CRASH** (Location $\rightarrow$ Crash)
3. **INVOLVED** (Crash $\rightarrow$ Vehicle) 
4. **ON_ROAD** (Crash $\rightarrow$ Road)
5. **HAPPENED_ON** (Crash $\rightarrow$ Time)
6. **KILLED_AT** (Person $\rightarrow$ Location)

In [79]:
# crash node
df_crash = df[['Crash ID', 'Crash Type', 'Number Fatalities']]
df_crash = df_crash.drop_duplicates()
df_crash.to_csv('nodes_relationships/node_crash.csv', index=False)
df_crash

,Crash ID,Crash Type,Number Fatalities
0,20241115,Single,1
1,20241125,Single,1
2,20246013,Single,1
3,20241002,Single,1
4,20243185,Single,1
...,...,...,...
10485,20144079,Single,1
10486,20145055,Single,1
10487,20144007,Single,1
10488,20145072,Single,1


In [80]:
# person node
df_person = df[['Gender', 'Age', 'Road User', 'Age Group']]
df_person = df_person.drop_duplicates().reset_index(drop=True)
df_person['Person ID'] = df_person.index + 1
df_person.to_csv('nodes_relationships/node_person.csv', index=False)
df_person

,Gender,Age,Road User,Age Group,Person ID
0,Male,74,Driver,65_to_74,1
1,Female,19,Driver,17_to_25,2
2,Female,33,Driver,26_to_39,3
3,Female,32,Driver,26_to_39,4
4,Female,61,Passenger,40_to_64,5
...,...,...,...,...,...
816,Female,64,Motorcycle rider,40_to_64,817
817,Female,43,Pedal cyclist,40_to_64,818
818,Male,86,Motorcycle rider,75_or_older,819
819,Female,49,Pedal cyclist,40_to_64,820


In [81]:
# killed relationship
df_killed = df.merge(df_person, on=['Gender', 'Age', 'Road User', 'Age Group'])
df_killed = df_killed[['Crash ID', 'Person ID']]
df_killed.to_csv('nodes_relationships/rel_killed.csv', index=False)
df_killed

,Crash ID,Person ID
0,20241115,1
1,20241125,2
2,20246013,3
3,20241002,4
4,20243185,5
...,...,...
10485,20144079,62
10486,20145055,187
10487,20144007,451
10488,20145072,278


In [82]:
# time node
df_time = df[['Month', 'Year', 'Dayweek', 'Time', 'Time of day', 'Day of week', 'Christmas Period', 'Easter Period']]
df_time = df_time.drop_duplicates().reset_index(drop=True)
df_time['Time ID'] = df_time.index + 1
df_time.to_csv('nodes_relationships/node_time.csv', index=False)
df_time

,Month,Year,Dayweek,Time,Time of day,Day of week,Christmas Period,Easter Period,Time ID
0,12,2024,Friday,4:00,Night,Weekday,Yes,No,1
1,12,2024,Friday,6:15,Day,Weekday,No,No,2
2,12,2024,Friday,9:43,Day,Weekday,Yes,No,3
3,12,2024,Friday,10:35,Day,Weekday,No,No,4
4,12,2024,Friday,13:00,Day,Weekday,No,No,5
...,...,...,...,...,...,...,...,...,...
9419,1,2014,Thursday,14:40,Day,Weekday,No,No,9420
9420,1,2014,Tuesday,16:42,Day,Weekday,No,No,9421
9421,1,2014,Tuesday,20:00,Night,Weekday,No,No,9422
9422,1,2014,Tuesday,21:30,Night,Weekday,No,No,9423


In [83]:
# happened_on relationship
df_happened_on = df.merge(df_time, on=['Month', 'Year', 'Dayweek', 'Time', 'Time of day', 'Day of week', 'Christmas Period', 'Easter Period'])
df_happened_on = df_happened_on[['Crash ID', 'Time ID']]
df_happened_on.to_csv('nodes_relationships/rel_happened_on.csv', index=False)
df_happened_on

,Crash ID,Time ID
0,20241115,1
1,20241125,2
2,20246013,3
3,20241002,4
4,20243185,5
...,...,...
10485,20144079,9420
10486,20145055,9421
10487,20144007,9422
10488,20145072,9423


In [84]:
# road node
df_road = df[['National Road Type', 'Speed Limit']]
df_road = df_road.drop_duplicates().reset_index(drop=True)
df_road['Road ID'] = df_road.index + 1
df_road.to_csv('nodes_relationships/node_road.csv', index=False)
df_road

,National Road Type,Speed Limit,Road ID
0,Arterial Road,100,1
1,Local Road,80,2
2,Local Road,50,3
3,National or State Highway,100,4
4,Sub-arterial Road,100,5
...,...,...,...
75,Sub-arterial Road,30,76
76,Arterial Road,10,77
77,Local Road,5,78
78,Collector Road,75,79


In [85]:
# on_road relationship
df_on_road = df.merge(df_road, on=['National Road Type', 'Speed Limit'])
df_on_road = df_on_road[['Crash ID', 'Road ID']]
df_on_road.to_csv('nodes_relationships/rel_on_road.csv', index=False)
df_on_road

,Crash ID,Road ID
0,20241115,1
1,20241125,2
2,20246013,3
3,20241002,4
4,20243185,4
...,...,...
10485,20144079,8
10486,20145055,16
10487,20144007,3
10488,20145072,8


In [86]:
# vehicle node
df_vehicle = df[['Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']]
df_vehicle = df_vehicle.drop_duplicates().reset_index(drop=True)
df_vehicle['Vehicle ID'] = df_vehicle.index + 1
df_vehicle.to_csv('nodes_relationships/node_vehicle.csv', index=False)
df_vehicle

,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Vehicle ID
0,No,No,No,1
1,No,No,Yes,2
2,Yes,No,Yes,3
3,No,Yes,No,4
4,No,Yes,Yes,5
5,Yes,No,No,6
6,Yes,Yes,No,7


In [87]:
# involved relationship
df_involved = df.merge(df_vehicle, on=['Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement'])
df_involved = df_involved[['Crash ID', 'Vehicle ID']]
df_involved.to_csv('nodes_relationships/rel_involved.csv', index=False)
df_involved

,Crash ID,Vehicle ID
0,20241115,1
1,20241125,1
2,20246013,1
3,20241002,1
4,20243185,1
...,...,...
10485,20144079,2
10486,20145055,1
10487,20144007,1
10488,20145072,1


In [88]:
# location node 
df_location = df[['State','SA4 Name 2021', 'National LGA Name 2024', 'National Remoteness Areas']]
df_location = df_location.drop_duplicates().reset_index(drop=True)
df_location['Location ID'] = df_location.index + 1
df_location.to_csv('nodes_relationships/node_location.csv', index=False)
df_location

,State,SA4 Name 2021,National LGA Name 2024,National Remoteness Areas,Location ID
0,NSW,Riverina,Wagga Wagga,Inner Regional Australia,1
1,NSW,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Inner Regional Australia,2
2,TAS,Launceston and North East,Northern Midlands,Inner Regional Australia,3
3,NSW,New England and North West,Armidale,Outer Regional Australia,4
4,QLD,Toowoomba,Lockyer Valley,Inner Regional Australia,5
...,...,...,...,...,...
781,WA,Western Australia - Wheat Belt,Gnowangerup,Outer Regional Australia,782
782,WA,Bunbury,Boyup Brook,Outer Regional Australia,783
783,WA,Perth - Inner,Cottesloe,Major Cities of Australia,784
784,WA,Western Australia - Wheat Belt,Wickepin,Remote Australia,785


In [89]:
# occurred_at relationship
df_occurred_at = df.merge(df_location, on=['State','SA4 Name 2021', 'National LGA Name 2024', 'National Remoteness Areas'])
df_occurred_at = df_occurred_at[['Crash ID', 'Location ID']]
df_occurred_at.to_csv('nodes_relationships/rel_occurred_at.csv', index=False)
df_occurred_at

,Crash ID,Location ID
0,20241115,1
1,20241125,2
2,20246013,3
3,20241002,4
4,20243185,5
...,...,...
10485,20144079,13
10486,20145055,542
10487,20144007,134
10488,20145072,617


In [90]:
# killed_at relationship
df_killed_at = df_occurred_at.merge(df_killed, on=['Crash ID'])
df_killed_at = df_killed_at.drop(columns='Crash ID')
df_killed_at.to_csv('nodes_relationships/rel_killed_at.csv', index=False)
df_killed_at

,Location ID,Person ID
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
12499,13,62
12500,542,187
12501,134,451
12502,617,278
